In [1]:
import pandas as pd

In [2]:
df = pd.read_json('tweets/coronavirus_massachusetts_tweets.txt', lines=True)

In [70]:
file_name = 'tweets/coronavirus_massachusetts_tweets.txt'

In [71]:
file_name[7:]

'coronavirus_massachusetts_tweets.txt'

In [55]:
df.iloc[1020]['entities']['hashtags']

[{'indices': [80, 88], 'text': 'COVID19'},
 {'indices': [89, 101], 'text': 'coronavirus'}]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94157 entries, 0 to 94156
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   contributors               0 non-null      float64            
 1   coordinates                227 non-null    object             
 2   created_at                 94157 non-null  datetime64[ns, UTC]
 3   display_text_range         94157 non-null  object             
 4   entities                   94157 non-null  object             
 5   favorite_count             94157 non-null  int64              
 6   favorited                  94157 non-null  bool               
 7   full_text                  94157 non-null  object             
 8   geo                        227 non-null    object             
 9   id                         94157 non-null  int64              
 10  id_str                     94157 non-null  int64              
 11  in

created_at = datetime64
display_text_range = list -> int64
favorite_count = int64
full_text = string
id_str = int64
possibly_sensitive = float64
retweet_count = int64

In [32]:
# new_df = df.drop(['contributors', 'display_text_range', 
#                   'favorite_count', 'favorited', 'id', 
#                   'in_reply_to_screen_name', 'in_reply_to_status_id',
#                   'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str',
#                   'possibly_sensitive', 'retweet_count', 'retweeted', 
#                   'lang', 'metadata', 'truncated', 'user'], axis=1)

new_df = df[['id_str', 'created_at', 'full_text', 'possibly_sensitive',
             'display_text_range', 'favorite_count', 'retweet_count']].copy()

In [33]:
new_df['character_count'] = new_df['display_text_range'].apply(lambda x: x[-1])

In [64]:
new_df = pd.concat([new_df, df['entities'].apply(pd.Series)], axis=1)

In [68]:
new_df['hashtags_count'] = new_df['hashtags'].apply(lambda x: len(x))
new_df['symbols_count'] = new_df['symbols'].apply(lambda x: len(x))
new_df['urls_count'] = new_df['urls'].apply(lambda x: len(x))
new_df['user_mentions_count'] = new_df['user_mentions'].apply(lambda x: len(x))

In [73]:
new_df.drop(['hashtags', 'symbols', 'urls', 
             'user_mentions', 'media'],
            axis=1, inplace=True)

In [76]:
new_df.columns

Index(['id_str', 'created_at', 'full_text', 'possibly_sensitive',
       'display_text_range', 'favorite_count', 'retweet_count',
       'character_count', 'hashtags_count', 'symbols_count', 'urls_count',
       'user_mentions_count'],
      dtype='object')

In [ ]:
new_df = df[['id_str', 'created_at', 'full_text',
             'entities', 'extended_entities',
             'coordinates', 'geo', 'place', 'source', 'is_quote_status',
             'quoted_status', 'quoted_status_id', 'quoted_status_id_str']].copy()

In [ ]:
new_df.possibly_sensitive.value_counts()

In [ ]:
import preprocessor as p
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
import re

In [ ]:
stop_words = stopwords.words('english')
# [stop_words.append(x.replace('\'', '')) for x in stop_words if "'" in x]
stop_words.extend(['coronavirus', 'covid', 'covidー', 'coronavirusoutbreak', 'coronaviruspandemic'])
stop_words = set(stop_words)
lemmatizer = WordNetLemmatizer()

In [ ]:
def cleanText(text):
    p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.EMOJI, p.OPT.SMILEY)
    preprocesser = lambda x: p.clean(x) #removes URL, @Mentions, Emojis, Smileys
    
    # prevent acronym for United States from losing meaning
    expand_us = lambda x: x.replace('U.S.', 'United States')
    
    # remove punctuation
    table = str.maketrans('', '', string.punctuation)
    remove_punctuation = lambda x: x.translate(table)
    
    # remove numbers
    text_nonum = lambda x: re.sub(r'\d+', '', x)
    
    # convert all letters to lowercase
    text_lower = lambda x: x.lower()

    # substitute multiple spaces with single space
    text_nospaces = lambda x: re.sub(r'\s+', ' ', x, flags=re.I)

    # remove all single characters
    text_single = lambda x: re.sub(r'\s+[a-zA-Z]\s+', ' ', x)
    
    # tokenize words
    tokenize = lambda x: word_tokenize(x)
    remove_stop = lambda x: [w for w in x if w not in stop_words]
#     lemmatize_tweet = lambda x: [lemmatizer.lemmatize(word) for word in x]
    create_string = lambda x: ' '.join(x)
    
    for function in [preprocesser, expand_us, remove_punctuation, 
                     text_nonum, text_lower, text_nospaces, 
                     text_single,
                     tokenize, remove_stop, 
                     # lemmatize_tweet, 
                     create_string
                    ]:
        text = text.map(function)
    return text

In [ ]:
new_df['full_text'] = cleanText(new_df.full_text)

In [ ]:
new_df

In [ ]:
# df.iloc[10].full_text

In [ ]:
# new_df.iloc[10].full_text

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans 

import umap.umap_ as umap
from yellowbrick.text import UMAPVisualizer

import matplotlib.pyplot as plt

In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '", topic_names[ix], "'")
        print(", ".join([feature_names[i]
                         for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
index_names = [f'component_{x}' for x in range(5)]

In [ ]:
# tf, dtm = createCustomTFIDFvectorizer(new_df, n_range=(1,2), stop = stop_words)

# don't use TF-IDF for tweets
# use CV because it's a small doc
# suggestion - binary = True ~ can throw in a lot of noise
cv = CountVectorizer(ngram_range = (1,1), stop_words = stop_words, min_df=3, max_df=0.85)
data_dtm = cv.fit_transform(new_df.full_text)
data_dtm = pd.DataFrame(data_dtm.toarray(), columns=cv.get_feature_names())
data_dtm.index = new_df.index

In [ ]:
data_dtm

In [ ]:
nmf_model = NMF(n_components = 5, random_state = 42)
doc_topic_nmf = nmf_model.fit_transform(data_dtm)

topic_word_nmf = pd.DataFrame(nmf_model.components_.round(3),
                                  index=index_names,
                                  columns=cv.get_feature_names())

# doc_topic_nmf_chosen = performCustomNMF(tf, dtm, index_names = index_names)
#len(doc_topic_nmf_chosen)

In [ ]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = cv.get_feature_names()
    
    word_dict = {};
    for i in range(5):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

get_nmf_topics(nmf_model, 20)

In [ ]:
topic_values = nmf_model.transform(data_dtm)
new_df['Topic'] = topic_values.argmax(axis=1)

In [ ]:
topic_word_nmf

In [ ]:
display_topics(nmf_model, cv.get_feature_names(), 60)

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42).fit(doc_topic_nmf)
cluster_topic_nmf = kmeans.labels_

In [ ]:
plt.figure(figsize=(15,10))
docs = doc_topic_nmf
labels = cluster_topic_nmf

umap = UMAPVisualizer(metric ='euclidean', colormap = 'Set1', random_state=42)
umap.fit(docs, labels)
umap.show()

In [ ]:
def performCustomNMF(vec, dtm, index_names = index_names):
    nmf_model = NMF(n_components = 10, random_state = 42)
    doc_topic_nmf = nmf_model.fit_transform(dtm)
    
    print('The mean of individual topic weights over all documents is:')
    mean = np.mean(doc_topic_nmf, axis=0)
    print(mean)
    
    print('---')
    print('The variance of individual topic weights over all documents is:')
    variance = np.var(doc_topic_nmf, axis=0)
    print(variance)
    
    topic_word_nmf = pd.DataFrame(nmf_model.components_.round(3),
                                  index=index_names,
                                  columns=vec.get_feature_names())
    print('---')
    print('The topics are:')
    display_topics(nmf_model, vec.get_feature_names(), 60)
    
    return_doc_topic_nmf = doc_topic_nmf.copy()
    kmeans = KMeans(n_clusters=num_features, random_state=random_state).fit(doc_topic_nmf)
    cluster_topic_nmf = kmeans.labels_
    best_topic_nmf = [x.argmax() for x in doc_topic_nmf]
    
    # UMAP Visualization Option 1 -------------------------------------------
    plt.figure(figsize=(15,10))
    docs = doc_topic_nmf
    labels = best_topic_nmf

    umap = UMAPVisualizer(metric ='euclidean', colormap = 'Set1', random_state=random_state)
    umap.fit(docs, labels)
    umap.show()

In [ ]:
def clean_text_one(docs):
    """
    Cleans tweet text so that it is in a form suitable for topic modeling.
    ---
    :param docs: Series of documents to be processed.
    :return: Series of processed texts.
    """

    # remove URLs and hyperlinks
    text_nourl = lambda x: re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|' \
                                  '(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x)

    # remove @ names
    text_noname = lambda x: re.sub('(@[A-Za-z0-9_]+)', '', x)

    # remove hashtags
    text_nohash = lambda x: re.sub('(#[A-Za-z0-9_]+)', '', x)

    # remove numbers
    text_nonum = lambda x: re.sub(r'\d+', '', x)

    # remove the new line character
    text_nonewline = lambda x: re.sub('\n', '', x)

    # remove punctuation
    text_nopunct = lambda x: ''.join([char for char in x if char not in string.punctuation])

    # convert all letters to lowercase
    text_lower = lambda x: x.lower()

    # substitute multiple spaces with single space
    text_nospaces = lambda x: re.sub(r'\s+', ' ', x, flags=re.I)

    # remove all single characters
    text_single = lambda x: re.sub(r'\s+[a-zA-Z]\s+', ' ', x)

    # apply all cleaning functions to input text
    for clean_func in [text_nourl, text_noname, text_nohash, text_nonum, text_nonewline, \
                       text_nopunct, text_lower, text_nospaces, text_single]:
        docs = docs.map(clean_func)

    return docs


def clean_text_two(docs):
    """
    ---
    :param docs: Pandas series of texts to pre-process.
    :return: Pandas series of cleaned text.
    """

    # remove repeating letters
    counter = 0
    ascii_lowercase = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
                      'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    # replace 'aaaaaaaahhhhh' with 'aahh'
    for letter in ascii_lowercase:
        for row_idx, doc in enumerate(docs):
            for word_idx, word in enumerate(doc):
                original_word = word
                while word != word.replace(letter*3, letter*2):
                    word = word.replace(letter*3, letter*2)
                    docs[row_idx][word_idx] = word
    return docs


def clean_text_three(docs):
    """
    ---
    :param docs: Pandas series of texts to pre-process.
    :return: Pandas series of cleaned text.
    """
    # Setting standard english stopwords + custom stopwords
    STOPWORDS = set(stopwords.words('english'))
    STOPWORDS.remove("not")
    stop_words = ['samsung', 'samsungs', 'galayxs', 'galaxy', 's ', ' s', 'plus', 'ultra', 'z', 'flip', 'unpacked']

    clean_text = []
    for tweet in docs:
        new_tweet = []
        for word in tweet.split():
            if (word not in STOPWORDS) and (word not in stop_words) and (word not in string.punctuation):
                new_tweet.append(word)
        clean_text.append(' '.join(new_tweet))

    return clean_text


def clean_text_four(docs):
    """
    Cleans tweet text so that it is in a form suitable for topic modeling.
    ---
    :param docs: Series of documents to be processed.
    :return: Series of processed texts.
    """

    # lemmatize tweets
    wordNetLemmatizer = WordNetLemmatizer()

    lemmatized_tweets = []
    for text in docs:
        try:
            lemmatized_tweets.append(wordNetLemmatizer.lemmatize(text))
        except:
            pass

    return lemmatized_tweets